# 
# 이전 Lipid Profile 추가하고 Categorization

In [1]:

##
import numpy as np
import pandas as pd
from import_data import *
import torch
import torch.optim as optim
import torch.nn as nn
from network import DNN, DNN_v1, DNN_cat
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from utils import EarlyStopping, KFOLD_GCN
# from matplotlib import pyplot as plt
# from config import args
from collections import Counter



if torch.cuda.is_available():
    print("Let's go CUDA!!!!!")
    cuda = torch.device('cuda')
else:
    print("No CUDA,,,")
    cuda = torch.device('cpu')
    
    
np.random.seed(777)
kfold = 5
# y_value = args.y

data1, data2, data3, data4 = DATA_PREPROCESS_SH_v2()

/home/kicarus/anaconda3/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
/home/kicarus/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (5,6,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


No CUDA,,,


In [2]:
xdata, ydata = xysplit(data2)
xdata = np.array(xdata)
ydata = np.array(ydata)

In [3]:
ckpath = 'checkpoint_wlipid_cat.pt'

In [4]:


# data, xdata, ydata = DATA_PREPROCESS_SH(log=False)
xdata = NORMALIZATION(xdata, 'standard')
ydata = CATEGORIZATION(ydata)
for i in range(len(ydata[0])):
    print(Counter(ydata[:, i]))


def _ychoice(y):
    this = np.transpose(np.concatenate((np.transpose(xdata), [ydata[:, y]]), axis=0))
    this = np.array(this)
    # this = this.astype(float)
    np.random.shuffle(this)
    return this
# data.to_csv('kky1.csv', index=None)





# def _TData(train, y):
#     """
#      0~14 features, 15~18 labels
#     """
#     til = size[1] - 4
#     train_dataset = TensorDataset(torch.Tensor(train[:, :til]).to(cuda),
#                                   torch.Tensor(train[:, y]).to(cuda))
#
#     return train_dataset

def _TData_v2(train):
    train_dataset = TensorDataset(torch.Tensor(train[:, :-1]).to(cuda),
                                  torch.LongTensor(train[:, -1]).to(cuda))

    return train_dataset



def Parameters(net):
    # criterion = nn.CrossEntropyLoss()
    criterion = nn.MSELoss(reduction='mean')
    # optimizer = optim.SGD(net.parameters(), lr=0.000001)
    optimizer = optim.Adam(net.parameters(), lr=0.01)

    return criterion, optimizer

def Parameters_cat(net):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    return criterion, optimizer


Counter({0: 3061, 1: 1758, 2: 563})
Counter({0: 4831, 1: 340, 2: 211})
Counter({2: 3048, 1: 2070, 0: 264})
Counter({0: 1942, 1: 1801, 2: 1191, 3: 367, 4: 81})


# 
# 
# 통계량 측정


In [5]:
def Learning_all(train_loader, val_loader, nodes, num, patience=10, num_classes=None):
    """
        train_loader: 학습시킬 데이터셋 (DataLoader 형식)
        val_loader: 평가할 데이터셋 (DataLoader 형식)
        num: Epochs

    """
    net = DNN_cat(xdata.shape[1], num_classes, nodes)
    criterion, optimizer = Parameters_cat(net)
    net.to(cuda)

    """Early Stopping Template"""
    for epoch in range(num):
        torch.cuda.empty_cache()
        if epoch == 0:
            early_stopping = EarlyStopping(patience=patience, path=ckpath, verbose=False)
        else:
            early_stopping = EarlyStopping(patience=patience, best_score=best_score, counter=counter, path=ckpath, verbose=False)

        # Training
        net.train()
        running_loss = 0.
        lcnt = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs = inputs.to(cuda)
            labels = labels.to(cuda)

            outputs = net(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            lcnt += len(labels)

        # Validation
        net.eval()
        net.to(cuda)

        val_loss = 0
        vcnt = 0
        for i, data in enumerate(val_loader):
            inputs, labels = data
            inputs = inputs.to(cuda)
            labels = labels.to(cuda)
            outputs = net(inputs)

            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)

            val_loss += loss.item()
            vcnt += len(labels)

        # Template
        best_score, counter, finish = early_stopping(val_loss / vcnt, net)
        # if epoch % 50 == 0:
        #     print('Epoch Now: %d' % epoch)

        if finish:
            break

    net1 = DNN_cat(xdata.shape[1], num_classes, nodes)
    net1.load_state_dict(torch.load(ckpath))
    net1.eval()

    # output save
    val_loss = 0
    correct = 0
    class_correct = list(0. for i in range(num_classes))
    class_total = list(0. for i in range(num_classes))
    precision_c = list(0. for i in range(num_classes))
    precision_t = list(0. for i in range(num_classes))
    label_all = []
    output_all = []
    
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            inputs, labels = data
            inputs = inputs.to(cuda)
            labels = labels.to(cuda)

            outputs = net1(inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs.data, 1)
            val_loss += loss.item()

            correct += (predicted == labels).sum().item()

            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

            for i in range(len(predicted)):
                output = predicted[i].item()
                precision_c[output] += c[i].item()
                precision_t[output] += 1
            
            label_all.append(labels)
            output_all.append(outputs)
            
        print('Best Epoch: %5d / Last Validation Loss: %5f / Accuracy: %.3f'
              % (epoch - patience, val_loss / vcnt, 100 * correct / vcnt))

        for cls in range(num_classes):
            if class_total[cls] == 0:
                print('To small dataset for %s' % cls)
            else:
                print('Recall of %5s : %2d %%' % (
                    cls, 100 * class_correct[cls] / class_total[cls]))

        for cls in range(num_classes):
            if precision_t[cls] == 0:
                print('No prediction for %s' % cls)
            else:
                print('Precision of %5s : %2d %%' % (
                    cls, 100 * precision_c[cls] / precision_t[cls]))

    return correct / vcnt, label_all, output_all


def RUN_all(batch_size, kfold, max_epoch, y_value, node_list, patience):
    # Train / Test split
    data = _ychoice(y_value)
    
    num_classes = len(set(data[:, -1]))
    tmp = KFOLD_GCN(data, KFOLD=kfold)
    train = np.vstack(tmp[:-1])
    test = tmp[-1]


    print('\n\n\n가즈아~!~!~!~!~!')
    print('y_value: %2d' % y_value)

    acc_box = []
    for nodes in node_list:

        acc = 0
        print('Nodes: %2d' % nodes)

        kfold_all = KFOLD_GCN(train, kfold)
        for k in range(kfold):
            validation = kfold_all[k]
            training = []
            for exp in range(kfold):
                if exp != k:
                    training.append(kfold_all[exp])
            training = np.vstack(training)

            train_loader = DataLoader(_TData_v2(training), batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(_TData_v2(validation), batch_size=batch_size, shuffle=False)
            acc_all, label_all, output_all = Learning_all(train_loader, val_loader, nodes, max_epoch, patience, num_classes)
            acc += acc_all

        print('Validation Accuracy: %f' % (acc / kfold))
        acc_box.append(acc / kfold)
        
        
    best_node = node_list[acc_box.index(max(acc_box))]
    train_loader = DataLoader(_TData_v2(train), batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(_TData_v2(test), batch_size=batch_size, shuffle=False)
    
    print('')
    print('This is best result')
    print('')
    acc_all, label_all, output_all = Learning_all(train_loader, test_loader, best_node, max_epoch, patience, num_classes)

    print('# of node: %d \nTest Accuracy: %f' %
          (best_node, acc_all))

    return np.hstack([i.numpy() for i in label_all]), np.vstack([i.numpy() for i in output_all])


# 
# Parameters

In [6]:
batch_size = 128
kfold = 5
max_epoch = 1000
n_list = [6, 9, 12, 15, 18, 21, 24, 27, 30]


# 
# 실행 / AUROC(임시)

In [7]:
a, b = RUN_all(batch_size, kfold, max_epoch, y_value=-4, node_list=n_list, patience=100)

/home/kicarus/anaconda3/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)





가즈아~!~!~!~!~!
y_value: -4
Nodes:  6
Early Stopping Validated
Best Epoch:   261 / Last Validation Loss: 0.006307 / Accuracy: 69.070
Recall of     0 : 83 %
Recall of     1 : 57 %
Recall of     2 : 25 %
Precision of     0 : 75 %
Precision of     1 : 57 %
Precision of     2 : 60 %
Early Stopping Validated
Best Epoch:   144 / Last Validation Loss: 0.006263 / Accuracy: 66.744
Recall of     0 : 80 %
Recall of     1 : 57 %
Recall of     2 : 21 %
Precision of     0 : 75 %
Precision of     1 : 53 %
Precision of     2 : 54 %
Early Stopping Validated
Best Epoch:    60 / Last Validation Loss: 0.006081 / Accuracy: 67.712
Recall of     0 : 83 %
Recall of     1 : 56 %
Recall of     2 : 16 %
Precision of     0 : 77 %
Precision of     1 : 52 %
Precision of     2 : 51 %
Early Stopping Validated
Best Epoch:    62 / Last Validation Loss: 0.005794 / Accuracy: 68.060
Recall of     0 : 82 %
Recall of     1 : 58 %
Recall of     2 : 18 %
Precision of     0 : 78 %
Precision of     1 : 52 %
Precision of     2 

In [8]:
a1, b1 = RUN_all(batch_size, kfold, max_epoch, y_value=-3, node_list=n_list, patience=100)




가즈아~!~!~!~!~!
y_value: -3
Nodes:  6


/home/kicarus/anaconda3/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Early Stopping Validated
Best Epoch:    40 / Last Validation Loss: 0.003481 / Accuracy: 89.872
Recall of     0 : 100 %
Recall of     1 :  0 %
Recall of     2 :  0 %
Precision of     0 : 89 %
No prediction for 1
No prediction for 2
Early Stopping Validated
Best Epoch:    95 / Last Validation Loss: 0.003123 / Accuracy: 90.581
Recall of     0 : 99 %
Recall of     1 :  0 %
Recall of     2 : 39 %
Precision of     0 : 91 %
Precision of     1 :  0 %
Precision of     2 : 54 %
Early Stopping Validated
Best Epoch:    67 / Last Validation Loss: 0.003011 / Accuracy: 89.895
Recall of     0 : 100 %
Recall of     1 :  1 %
Recall of     2 :  0 %
Precision of     0 : 90 %
Precision of     1 : 25 %
No prediction for 2
Early Stopping Validated
Best Epoch:    19 / Last Validation Loss: 0.003411 / Accuracy: 89.675
Recall of     0 : 100 %
Recall of     1 :  0 %
Recall of     2 :  0 %
Precision of     0 : 89 %
No prediction for 1
No prediction for 2
Early Stopping Validated
Best Epoch:    66 / Last Validatio

In [9]:
a2, b2 = RUN_all(batch_size, kfold, max_epoch, y_value=-2, node_list=n_list, patience=100)




가즈아~!~!~!~!~!
y_value: -2
Nodes:  6


/home/kicarus/anaconda3/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Early Stopping Validated
Best Epoch:    83 / Last Validation Loss: 0.004556 / Accuracy: 75.233
Recall of     0 : 19 %
Recall of     1 : 68 %
Recall of     2 : 84 %
Precision of     0 : 72 %
Precision of     1 : 67 %
Precision of     2 : 80 %
Early Stopping Validated
Best Epoch:    65 / Last Validation Loss: 0.004251 / Accuracy: 76.860
Recall of     0 :  2 %
Recall of     1 : 69 %
Recall of     2 : 88 %
Precision of     0 : 100 %
Precision of     1 : 70 %
Precision of     2 : 80 %
Early Stopping Validated
Best Epoch:    57 / Last Validation Loss: 0.004593 / Accuracy: 74.216
Recall of     0 :  2 %
Recall of     1 : 71 %
Recall of     2 : 82 %
Precision of     0 : 25 %
Precision of     1 : 65 %
Precision of     2 : 81 %
Early Stopping Validated
Best Epoch:    67 / Last Validation Loss: 0.004208 / Accuracy: 77.352
Recall of     0 :  0 %
Recall of     1 : 77 %
Recall of     2 : 84 %
No prediction for 0
Precision of     1 : 68 %
Precision of     2 : 84 %
Early Stopping Validated
Best Epoch: 

In [10]:
a3, b3 = RUN_all(batch_size, kfold, max_epoch, y_value=-1, node_list=n_list, patience=100)




가즈아~!~!~!~!~!
y_value: -1
Nodes:  6


/home/kicarus/anaconda3/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Early Stopping Validated
Best Epoch:   274 / Last Validation Loss: 0.009118 / Accuracy: 53.380
Recall of     0 : 73 %
Recall of     1 : 47 %
Recall of     2 : 47 %
Recall of     3 :  6 %
Recall of     4 :  0 %
Precision of     0 : 66 %
Precision of     1 : 47 %
Precision of     2 : 41 %
Precision of     3 : 30 %
No prediction for 4
Early Stopping Validated
Best Epoch:   200 / Last Validation Loss: 0.009092 / Accuracy: 55.180
Recall of     0 : 71 %
Recall of     1 : 55 %
Recall of     2 : 46 %
Recall of     3 :  5 %
Recall of     4 :  0 %
Precision of     0 : 69 %
Precision of     1 : 50 %
Precision of     2 : 42 %
Precision of     3 : 25 %
No prediction for 4
Early Stopping Validated
Best Epoch:   754 / Last Validation Loss: 0.008650 / Accuracy: 55.517
Recall of     0 : 69 %
Recall of     1 : 55 %
Recall of     2 : 50 %
Recall of     3 : 10 %
Recall of     4 :  0 %
Precision of     0 : 70 %
Precision of     1 : 49 %
Precision of     2 : 44 %
Precision of     3 : 42 %
No prediction for 

In [11]:
f1 = (a, b)
f2 = (a1, b1)
f3 = (a2, b2)
f4 = (a3, b3)

import pickle

filelist = [f1, f2, f3, f4]
filename = ['TCH', 'TG', 'HDL', 'LDL']

for i in range(4):
    name = filename[i] + '_cat.pkl'
    with open(name, 'wb') as f:
        pickle.dump(filelist[i], f)
        

# "
# 일단 여기까지
# "

In [ ]:
np.array([np.argmax(i) for i in b4])

In [ ]:
np.array([softmax(i) for i in b4])

In [16]:
from sklearn.metrics import roc_auc_score
from scipy import special


/bin/sh: 1: source: not found


In [17]:
from sklearn.metrics import roc_auc_score


roc_auc_score(a, np.array([softmax(i) for i in b]), multi_class='ovr')

AttributeError: module 'scipy.special' has no attribute 'softmax'

In [ ]:
roc_auc_score(a4, np.array([np.argmax(i) for i in b4]), multi_class='ovr')

In [ ]:
roc_auc_score(a5, np.array([np.argmax(i) for i in b5]), multi_class='ovr')

In [ ]:
roc_auc_score(a6, np.array([np.argmax(i) for i in b6]), multi_class='ovr')

In [ ]:
roc_auc_score(a7, np.array([np.argmax(i) for i in b7]), multi_class='ovr')

In [ ]:
df_sort_2 = pd.merge(df_sort, data2, on='RAND_KEY').drop('RAND_KEY', axis=1).dropna(axis=0)
data = df_sort_2.copy()
xdata = df_sort_2[df_sort_2.columns.difference(['L3008_cholesterol', 'L3061_tg', 'L3062_hdl', 'L3068_ldl'])]
ydata = np.array(df_sort_2[['L3008_cholesterol', 'L3061_tg', 'L3062_hdl', 'L3068_ldl']])


# data, xdata, ydata = DATA_PREPROCESS_SH(log=False)
xdata = NORMALIZATION(xdata, 'standard')
ydata = CATEGORIZATION_v2(ydata)
for i in range(len(ydata[0])):
    print(Counter(ydata[:, i]))


def _ychoice(y):
    this = np.transpose(np.concatenate((np.transpose(xdata), [ydata[:, y]]), axis=0))
    this = np.array(this)
    # this = this.astype(float)
    np.random.shuffle(this)
    return this

In [ ]:
a4, b4 = RUN_all(batch_size, kfold, max_epoch, y_value=-4, node_list=n_list, patience=100)

In [ ]:
a5, b5 = RUN_all(batch_size, kfold, max_epoch, y_value=-3, node_list=n_list, patience=100)

In [ ]:
a6, b6 = RUN_all(batch_size, kfold, max_epoch, y_value=-2, node_list=n_list, patience=100)

In [ ]:
a7, b7 = RUN_all(batch_size, kfold, max_epoch, y_value=-1, node_list=n_list, patience=100)